In [240]:
# Importamos las librerías necesarias para la limpieza
import pandas as pd
import numpy as np
import re

In [241]:
# Cargamos el dataset original (sucio) desde la carpeta data
df=pd.read_csv('../data/dataset_sucio_para_limpieza.csv')

# Mostramos algunas filas para comprobar que se ha cargado bien
df.head(3)

,Company Names!!,Car_Name @,Tipo de motor,CC / Batt.Capacidad,Horse Power(HP)!,velocidad max kmh,0_100 km/h (seg),Precio $$ coche,TipoCombustible,Num Asientos,Par motor (Nm)??
0,Ferrari,SF90 STRADALE,V8,3990cc,963hp,340km/h,2.5 sec,"$1,100,000",Plug In Hybrid,2,800 Nm
1,Rolls Royce,PHANTOM,V12,6749cc,563-hp,250km/h,5.3 sec,"$460,000",Petrol,5,900 N·m
2,Ford,KA+,1.2L Petrol,"1,200cc",70-85hp,165km/h,10.5 sec,"$12,000-$15,000",Petrol,5,100 - 140 Nm


In [242]:
# Creamos una copia del dataset original para trabajar sobre ella
df_clean = df.copy()

# Comprobamos que podemos acceder a la copia.
df_clean.sample(2)

,Company Names!!,Car_Name @,Tipo de motor,CC / Batt.Capacidad,Horse Power(HP)!,velocidad max kmh,0_100 km/h (seg),Precio $$ coche,TipoCombustible,Num Asientos,Par motor (Nm)??
1294,Nissan,Z NISMO,3.0L Twin-Turbo V6,2997cc,420hp,250km/h,4.0 sec,$68000,Petrol,2,520 Nm
649,Tata Motors,Nexon EV Max,Permanent Magnet Synchronous,40.5kWh,143hp,140km/h,9 sec,25.000€,Electric,5,250 Nm


### 1.Introducción

En este notebook voy a trabajar con el dataset después de haber realizado la fase de exploración. En la exploración ya detecté varios problemas importantes: columnas con texto mezclado con números, unidades inconsistentes, símbolos como $ o cc, errores ortográficos, valores faltantes, formatos diferentes para la misma categoría y, en general, datos bastante desordenados.

El objetivo de este notebook es limpiar y estandarizar todo el dataset para dejarlo listo para análisis reales. Esto incluye:

- Renombrar las columnas.

- Corregir formatos numéricos (hp, cc, velocidad, precio...)

- Convertir textos como “dos” o “cuatro” en números

- Normalizar categorías (petrol, diesel...)

- Eliminar símbolos y unidades que estorben (en algunos nombres de columnas podemos observar $ o simbolos extraños)

- Tratar valores nulos y errores

El resultado final será un nuevo archivo: dataset_clean.csv, que usaré después para las visualizaciones y el análisis final.

### 2.Renombrar columnas

In [243]:
#Vamos a ver los nombres de las columnas para poder renombrarlos correctamente
df.columns

Index([' Company  Names!!', 'Car_Name @', 'Tipo de motor  ',
       'CC / Batt.Capacidad ', ' Horse Power(HP)! ', 'velocidad max kmh ',
       '0_100 km/h  (seg) ', 'Precio $$ coche', 'TipoCombustible  ',
       '  Num Asientos ', 'Par motor (Nm)??'],
      dtype='object')

In [244]:
# Diccionario con nombres sucios --> nombres limpios
column_renaming = {
    ' Company  Names!!': 'company_name',
    'Car_Name @': 'car_name',
    'Tipo de motor  ': 'engine_type',
    'CC / Batt.Capacidad ': 'cc_or_battery',
    ' Horse Power(HP)! ': 'horse_power',
    'velocidad max kmh ': 'top_speed',
    '0_100 km/h  (seg) ': 'acceleration_0_100',
    'Precio $$ coche': 'price',
    'TipoCombustible  ': 'fuel_type',
    '  Num Asientos ': 'seats',
    'Par motor (Nm)??': 'torque_nm'
}

# Aplicamos el renombrado
df_clean = df_clean.rename(columns=column_renaming)

# Comprobamos el resultado
df_clean.columns


Index(['company_name', 'car_name', 'engine_type', 'cc_or_battery',
       'horse_power', 'top_speed', 'acceleration_0_100', 'price', 'fuel_type',
       'seats', 'torque_nm'],
      dtype='object')

In [245]:
df_clean.head(2)

,company_name,car_name,engine_type,cc_or_battery,horse_power,top_speed,acceleration_0_100,price,fuel_type,seats,torque_nm
0,Ferrari,SF90 STRADALE,V8,3990cc,963hp,340km/h,2.5 sec,"$1,100,000",Plug In Hybrid,2,800 Nm
1,Rolls Royce,PHANTOM,V12,6749cc,563-hp,250km/h,5.3 sec,"$460,000",Petrol,5,900 N·m


Una vez realizada la limpieza de los nombres, vamos a continuar con la limpieza de los valores de cada columna.

### 3.Limpiar valores de las columnas

In [246]:
df_clean.columns

Index(['company_name', 'car_name', 'engine_type', 'cc_or_battery',
       'horse_power', 'top_speed', 'acceleration_0_100', 'price', 'fuel_type',
       'seats', 'torque_nm'],
      dtype='object')

Para realizar la limpieza de datos correctamente, voy a trabajar cada columna de manera individual.

- Para ello, crearé una función específica para cada columna.
- A cada función se le pasará como argumento el valor de la columna correspondiente.
- Dentro de cada función aplicaré las transformaciones necesarias según las características de esa columna.

### 3.1 Limpiar columna horsepower

In [247]:
def limpiar_horsepower(col):
    # Convierto los valores a string para poder trabajar con ellos
    col = col.astype(str)

    # Extrae todos los números de cada valor, IMP poner expand=False para que devuelva una Serie.
    col = col.str.extract(r"(\d+)", expand=False)

    # Convertir a número entero, errores → NaN
    col = pd.to_numeric(col, errors="coerce").astype("Int64")

    return col

df_clean['horse_power'] = limpiar_horsepower(df_clean['horse_power'])

In [248]:
df_clean["horse_power"].sample(2)

5      710
123    591
Name: horse_power, dtype: Int64

### 3.2 Limpiar columna cc_or_battery

Según el análisis previo, he decidido conservar únicamente los valores correspondientes a la cilindrada dentro de esta columna. Todos los demás valores que no representen una cilindrada válida se reemplazarán por valores nulos.

In [249]:
def limpiar_engine_cc(col):
    #Lista donde guardo los resultados
    resultado = []

    #Voy a recorrer cada valor de la columna.
    for valor in col:
        
        # Convierto a string en minúscula y sin espacios.
        v = str(valor).lower().strip()

        # Como bien he explicado antes me quedo solo con la cilindrada lo demás NaN.
        if "kwh" in v or "battery" in v or "batt" in v or v in ["", "-", "nan"]:
            resultado.append(np.nan)
            continue

        # Uso una expresión regular para extraer los números del string. Uso replace para quitar comas y espacios internos.
        numeros = re.findall(r"\d+", v.replace(" ", "").replace(",", ""))

        #Si no hay números, NaN
        if not numeros:
            resultado.append(np.nan)
            continue

        # En caso de tener más numero solo me quedo con el primero.
        num = int(numeros[0])

        # Hago este filtro para evitar valores irreales
        if 300 <= num <= 8000:
            resultado.append(num)
        else:
            resultado.append(np.nan)

    # Devuelvo Serie con tipo entero ya que Int64 permiter tener enteros y NaN.
    return pd.Series(resultado, dtype="Int64")


`IMP`

Voy a renombrar la columna ya que no tiene mucho sentido que siga mostrando la bateria en el nombre.

In [250]:
#renombramos la columna cc_or_battery a engine_cc
df_clean = df_clean.rename(columns={"cc_or_battery": "engine_cc"}) 

#Aplico la función de limpieza a la columna engine_cc
df_clean["engine_cc"] = limpiar_engine_cc(df_clean["engine_cc"])

#Muestro un ejemplo de los valores limpios
df_clean["engine_cc"].sample(5)


1222    3982
44      5204
709     1500
1266    <NA>
534     <NA>
Name: engine_cc, dtype: Int64

### 3.3 Limpiar columna top_speed

In [251]:
def limpiar_top_speed(col):
    
    # Convertir todo a string
    limpieza=(col.astype(str)                          
    .str.extract(r'(\d+)')           # Extrae el primer número que encuentre
    .astype('Int64'))               # Convierte a entero
    
    return limpieza

df_clean['top_speed'] = limpiar_top_speed(df_clean['top_speed'])

In [252]:
#Muestro un ejemplo de los valores limpios y además comprueba el tipo de dato
df_clean["top_speed"].sample(5)

892     110
205     180
1167    200
344     190
1175    160
Name: top_speed, dtype: Int64

### 3.4 Limpiar columna aceleración

In [253]:
def limpiar_acceleration(col):

    limpieza=(col.astype(str)                 # Convertimos todo a texto
           .str.extract(r'(\d+\.?\d*)')       # Usando la regex extraigo número entero o decimal
           .astype('float')   )               # Lo paso a float
    
    return limpieza

df_clean['acceleration_0_100'] = limpiar_acceleration(df_clean['acceleration_0_100'])

In [254]:
df_clean["acceleration_0_100"].isnull().sum()

np.int64(6)

### 3.5 Limpieza columna precio

En esta columna se extrae únicamente el primer número que aparezca en el texto.

Antes de extraerlo se eliminan símbolos y caracteres que no sirven (como $, €, puntos, guiones o texto).
Si no se encuentra ningún número, se devuelve NaN.

Finalmente, el valor obtenido se convierte a float.

In [255]:
def limpiar_price(col):

    # Convertir a string
    col = col.astype(str)

    # Quito los simbolos
    col = col.str.replace('$', '', regex=False) #Pongo regex=False 
    col = col.str.replace('€', '', regex=False)
    col = col.str.strip()

    # 2) Si hay rangos tipo "12000-15000" --> Me quedo con el primero
    col = col.str.split('-', n=1).str[0]

    # 3) Quitar puntos y comas
    col = col.str.replace('.', '', regex=False)
    col = col.str.replace(',', '', regex=False)

    # Extraer solo los números. Si no hay --> NaN
    col = col.str.extract(r'(\d+)', expand=False)

    # Convierto a entero
    return col.astype('Int64')

df_clean['price'] = limpiar_price(df_clean['price'])


In [256]:
df_clean["price"].sample(10)

1180     30000
929      80000
295       <NA>
709      26600
956      30000
244      50000
984      33000
1314     36000
328      45000
39      294000
Name: price, dtype: Int64

In [257]:
df_clean["price"].isnull().sum()

np.int64(163)

### 3.6 Limpieza columna seats

En esta columna aparecen algunos valores escritos como texto, por ejemplo "dos" en lugar de 2.
Para coches, lo habitual es que tengan entre 1 y 9 asientos (este último valor más común en furgonetas). 

Por eso convierto las palabras a números y descarto cualquier valor fuera de ese rango.

In [258]:
def limpiar_seats(col):

    # convertir a texto, poner en minúsculas y quitar espacios
    col = col.astype(str).str.lower().str.strip()

    resultado = []

    traducciones = {
        "uno": "1", "dos": "2", "tres": "3", "cuatro": "4",
        "cinco": "5", "seis": "6", "siete": "7",
        "ocho": "8", "nueve": "9"
    }

    for v in col:

        # Si es una palabra como "dos", convertirla a su número "2"
        if v in traducciones:
            v = traducciones[v]

        # Si el valor es un número válido entre 1 y 9, lo guardo
        if v in ["1","2","3","4","5","6","7","8","9"]:
            resultado.append(int(v))
        else:
            # Si no es válido, lo marco como NaN
            resultado.append(np.nan)

    # Devuelvo los valores limpios como columna de tipo entero
    return pd.Series(resultado, dtype="Int64")

df_clean['seats'] = limpiar_seats(df_clean['seats'])

In [259]:
df_clean["seats"].sample(10)

376     4
894     2
248     5
1172    5
672     7
1108    2
115     4
1017    5
288     7
625     4
Name: seats, dtype: Int64

### 3.7 Limpieza columna torque

In [260]:
def limpiar_torque(col):

    #Convierto a string y paso a minúsculas
    col = col.astype(str).str.lower().str.strip()

    # Extraigo los números usando regex
    numeros = col.str.extract(r"(\d+)", expand=False)

    # Al devolverlo lo convierto a entero
    return numeros.astype("Int64")

df_clean["torque_nm"] = limpiar_torque(df_clean["torque_nm"])

In [261]:
df_clean["torque_nm"].sample(5)

973     200
198     300
678     617
1381    340
962     400
Name: torque_nm, dtype: Int64

### 3.8 Limpieza columna fuel_type

In [262]:
def limpiar_fuel_type(col):

    # Convierto a string, paso a minúsculas, quito espacios y espacios extra internos usando regex
    col = col.astype(str).str.lower().str.strip().str.replace(r'\s+', ' ', regex=True)  
    
    
    resultado = []

    #Como tengo un monton de valores voy a normalizarlos
    for v in col:

        # En caso de que sean valores vacíos o nulos
        if v in ["", "nan", "-", "none"]:
            resultado.append(np.nan)
            continue

        # Clasifico en los grandes grupos que hay y si no other
        if "petrol" in v:
            resultado.append("petrol")

        elif "diesel" in v:
            resultado.append("diesel")

        elif "hybrid" in v or "plug" in v or "hyrbrid" in v:
            resultado.append("hybrid")

        elif "electric" in v or "ev" in v:
            resultado.append("electric")

        else:
            resultado.append("other")

    #devuelvo una serie ya que las operaciones las he almacenado en una lista
    return pd.Series(resultado)

df_clean["fuel_type"] = limpiar_fuel_type(df_clean["fuel_type"])


In [263]:
df_clean["fuel_type"].tolist()

['hybrid',
 'petrol',
 'petrol',
 'petrol',
 'petrol',
 'petrol',
 'petrol',
 'petrol',
 'petrol',
 'petrol',
 'petrol',
 'petrol',
 'petrol',
 'petrol',
 'petrol',
 'petrol',
 'petrol',
 'petrol',
 'petrol',
 'petrol',
 'diesel',
 'petrol',
 'petrol',
 'petrol',
 'petrol',
 'petrol',
 'petrol',
 'petrol',
 'petrol',
 'petrol',
 'petrol',
 'petrol',
 'hybrid',
 'hybrid',
 'petrol',
 'petrol',
 'petrol',
 'petrol',
 'petrol',
 'petrol',
 'petrol',
 'petrol',
 'hybrid',
 'petrol',
 'petrol',
 'petrol',
 'petrol',
 'petrol',
 'petrol',
 'petrol',
 'petrol',
 'petrol',
 'petrol',
 'petrol',
 'petrol',
 'petrol',
 'petrol',
 'petrol',
 'petrol',
 'petrol',
 'petrol',
 'petrol',
 'petrol',
 'petrol',
 'petrol',
 'petrol',
 'petrol',
 'petrol',
 'petrol',
 'petrol',
 'petrol',
 'petrol',
 'petrol',
 'petrol',
 'petrol',
 'petrol',
 'petrol',
 'petrol',
 'petrol',
 'petrol',
 'petrol',
 'petrol',
 'petrol',
 'petrol',
 'petrol',
 'petrol',
 'petrol',
 'petrol',
 'petrol',
 'petrol',
 'petrol',

In [264]:
df_clean["fuel_type"].unique()

array(['hybrid', 'petrol', 'diesel', 'electric', 'other'], dtype=object)

### 3.9 Limpieza columna marca y modelo

Tanto para la marca de coche como para el modelo voy a usar la misma función.

In [265]:
def limpiar_company_and_car_name(col):

    #convierto a string, paso a minúscula, quito espacios y reemplazo espacios internos por _
    col = col.astype(str).str.lower().str.strip().str.replace(r'\s+', '_', regex=True)   
    
    return col

In [266]:
# Aplico la función a las dos columnas
df_clean["company_name"] = limpiar_company_and_car_name(df_clean["company_name"])
df_clean["car_name"] = limpiar_company_and_car_name(df_clean["car_name"])

In [267]:
df_clean[["company_name", "car_name"]].sample(5)

,company_name,car_name
943,acura,ilx_premium
1144,mazda,bt-50
757,chevrolet,tahoe_premier
983,peugeot,3008_allure
351,volkswagen,golf_tdi


Como podemos observar tanto company_name como car_name se han normalizado correctamente

### 3.10 Limpieza columna engine_type

He normalizado los tipos de motor agrupando en unos grupos generales (electric, hybrid, boxer, V-engines, inline-engines y others). Así reduzco la gran variedad de formatos y hago más fácil el análisis.

In [268]:
def clean_engine_type(col):

    # Convierto a string, paso a minúscula y quito espacios
    col = col.astype(str).str.lower().str.strip()
    
    #guardo los nuevos valores en una lista
    new_col = []

    # Recorro cada valor de la columna
    for x in col:

        #dependiendo del tipo al que pertenezca lo clasifico
        if x in ("nan", ""):
            new_col.append(np.nan)
        elif "electric" in x:
            new_col.append("electric")
        elif "hybrid" in x or "phev" in x or "plug" in x:
            new_col.append("hybrid")
        elif "boxer" in x or "flat" in x:
            new_col.append("boxer")
        elif "v12" in x:
            new_col.append("v12")
        elif "v10" in x:
            new_col.append("v10")
        elif "v8" in x:
            new_col.append("v8")
        elif "v6" in x:
            new_col.append("v6")
        elif "i4" in x or "inline-4" in x or "4-cylinder" in x:
            new_col.append("i4")
        elif "i3" in x or "inline-3" in x or "3-cylinder" in x:
            new_col.append("i3")
        else:
            new_col.append("others")

    return pd.Series(new_col)

df_clean["engine_type"] = clean_engine_type(df_clean["engine_type"])

In [269]:
df_clean["engine_type"].sample(5)

229           i4
1195          i4
1305      hybrid
854     electric
1345          v6
Name: engine_type, dtype: object

### 4.Comprobación limpieza dataset

Una vez realizada la limpieza vamos a volver a examinarla poder sacar conclusiones reales del dataset seleccionado.

In [270]:
df_clean.head()

,company_name,car_name,engine_type,engine_cc,horse_power,top_speed,acceleration_0_100,price,fuel_type,seats,torque_nm
0,ferrari,sf90_stradale,v8,3990,963,340,2.5,1100000,hybrid,2,800
1,rolls_royce,phantom,v12,6749,563,250,5.3,460000,petrol,5,900
2,ford,ka+,others,1200,70,165,10.5,12000,petrol,5,100
3,mercedes,gt_63_s,v8,3982,630,250,3.2,161000,petrol,4,900
4,audi,r8_gt,v10,5204,602,320,3.6,253290,petrol,2,560


Todo el contenido está correctamente normalizado, con los nombres del dataset en minúscula, sin espacios extra ni caracteres raros.

Por otro lado podemos observar que a los valores de las columnas se le han aplicado unas funciones específicas para realizar una limpieza de cada una de ellas.

`Pero...¿Qué tipos de datos tiene cada columna?`

Para poder responder dicha pregunta nos vamos ayudar de:

In [271]:
df_clean.dtypes

company_name           object
car_name               object
engine_type            object
engine_cc               Int64
horse_power             Int64
top_speed               Int64
acceleration_0_100    float64
price                   Int64
fuel_type              object
seats                   Int64
torque_nm               Int64
dtype: object

Los tipos de datos son los adecuados:
- Las columnas categóricas aparecen como object.
- Las columnas numéricas se encuentran en formatos correctos (Int64, float64)

`Pero...¿Existen valores ilógicos?`

In [ ]:
df_clean.describe()

,engine_cc,horse_power,top_speed,acceleration_0_100,price,seats,torque_nm
count,1112.0,1324.0,1323.0,1384.000000,1227.0,1344.0,1363.0
mean,2968.538669,315.283988,217.668178,7.345303,18720037.301548,4.837054,443.354365
std,1643.054862,228.202426,53.116657,3.244472,229888138.410735,1.370173,294.254779
min,323.0,1.0,80.0,1.900000,4000.0,1.0,1.0
25%,1798.0,154.0,180.0,5.075000,29282.5,4.0,250.0
50%,2488.0,261.0,205.0,7.000000,45000.0,5.0,380.0
75%,3745.0,401.0,250.0,9.000000,77500.0,5.0,570.0
max,7993.0,2488.0,500.0,35.000000,4500050000.0,9.0,3500.0


Al analizar el resumen estadístico, la mayoría de las columnas tienen valores coherentes pero también existen algunos valores que no son reales, lo que indica que ciertas filas contienen datos erróneos.

Por ejemplo:
- El precio máximo alcanza un valor extremadamente alto (450.000.000), muy lejos de cualquier vehículo real del mercado, por lo que claramente se trata de un dato incorrecto.
- El torque mínimo aparece como 1 Nm, una cifra imposible para cualquier coche, ya que incluso los vehículos más pequeños superan ese valor.

Estos registros representan anomalías o errores de entrada de datos, por lo que será necesario revisarlos o eliminarlos para evitar que afecten al análisis y a las conclusiones posteriores.

### 5.Revisión valores incoherentes

In [ ]:
df_clean.describe()

,engine_cc,horse_power,top_speed,acceleration_0_100,price,seats,torque_nm
count,1112.0,1324.0,1323.0,1384.000000,1227.0,1344.0,1363.0
mean,2968.538669,315.283988,217.668178,7.345303,18720037.301548,4.837054,443.354365
std,1643.054862,228.202426,53.116657,3.244472,229888138.410735,1.370173,294.254779
min,323.0,1.0,80.0,1.900000,4000.0,1.0,1.0
25%,1798.0,154.0,180.0,5.075000,29282.5,4.0,250.0
50%,2488.0,261.0,205.0,7.000000,45000.0,5.0,380.0
75%,3745.0,401.0,250.0,9.000000,77500.0,5.0,570.0
max,7993.0,2488.0,500.0,35.000000,4500050000.0,9.0,3500.0


In [292]:
# Precio incoherente lo ponemos como NaN
df_clean.loc[df_clean["price"] > 10000000, "price"] = np.nan

# Torque incoherente lo ponemos como NaN
df_clean.loc[df_clean["torque_nm"] <= 150, "torque_nm"] = np.nan
df_clean.loc[df_clean["torque_nm"] >= 2000, "torque_nm"] = np.nan

# CV incoherente lo ponemos como NaN
df_clean.loc[df_clean["horse_power"] < 10, "horse_power"] = np.nan


In [293]:
df_clean.describe()

,engine_cc,horse_power,top_speed,acceleration_0_100,price,seats,torque_nm
count,1112.0,1320.0,1323.0,1384.000000,1216.0,1344.0,1244.0
mean,2968.538669,314.457576,217.668178,7.345303,124338.625822,4.837054,463.45418
std,1643.054862,219.95078,53.116657,3.244472,479281.447244,1.370173,245.732089
min,323.0,26.0,80.0,1.900000,4000.0,1.0,151.0
25%,1798.0,155.0,180.0,5.075000,29000.0,4.0,275.0
50%,2488.0,261.0,205.0,7.000000,45000.0,5.0,400.0
75%,3745.0,401.0,250.0,9.000000,75100.0,5.0,600.0
max,7993.0,1850.0,500.0,35.000000,9000000.0,9.0,1800.0


También podemos observar que un coche tiene más de 2000cv

In [294]:
df_clean[df_clean["horse_power"] >= 2000]

#Aquí podemos observar que no tiene lógica ya que la velocidad máxima es de 140.

,company_name,car_name,engine_type,engine_cc,horse_power,top_speed,acceleration_0_100,price,fuel_type,seats,torque_nm


In [295]:
df_clean.loc[df_clean["horse_power"] == 2488, "horse_power"] = 140

### 6. Conclusión tras realizar la limpieza

Después de eliminar los valores incoherentes, podemos observar que los datos resultantes describen principalmente vehículos de gama media-alta y alta.

La potencia media ronda los 300 caballos, lo que indica que el conjunto incluye coches con un rendimiento notablemente superior al de un vehículo estándar.

El precio medio se sitúa alrededor de los 124.000 €, una cifra elevada que se debe, en parte, a la presencia de modelos extremadamente caros como algunos superdeportivos, por ejemplo un Bugatti valorado en de 9 millones de euros, lo que incrementa la media general.

En conjunto, el dataset representa coches con prestaciones altas, orientados a un público exigente o modelos de segmentos superiores.
Con los datos ya limpios y coherentes, estamos en condiciones de avanzar a la fase de visualización.

`Lo único que nos queda antes de avanzar de fase es guardar nuestro nuevo dataset limpiado`

In [296]:
df_clean.to_csv('../data/dataset_limpio.csv', index=False)